### VaMSL: Approximate inference of $p(G, \Theta, C,  \pi | D)$ for mixtures of (non)linear Gaussian Bayes nets

VaMSL carries out approximate inference over mixtures of Bayesian networks (BN) using coordinate ascent variational inference (CAVI). We instantiate the CAVI update over BNs with the Differentiable Bayesian Structure Learning (DiBS) ([Lorch et al., 2021](https://proceedings.neurips.cc/paper/2021/hash/ca6ab34959489659f8c3776aaf1f8efd-Abstract.html)) framework and it's Stein Variational gradient descent (SVGD) ([Liu and Wang, 2016](https://proceedings.neurips.cc/paper/2016/hash/b3ba8f1bee1238a2f37603d90b58898d-Abstract.html)) implementation.

DiBS leverages a generative model for graphs $p(G | Z)$ conditional on the continuous latent variable $Z$ to enable the use of gradient optimization methods in the (otherwise discrete) graph search accross BNs. Their SVGD implementation enables Bayesian structure learning by yielding a posterior in the BN space $p(G, \Theta | D)$.

In [ ]:
%pip install ../. 
import jax
import jax.numpy as jnp
import jax.random as random

print(f"JAX backend: {jax.default_backend()}")
key=random.PRNGKey(123)

### Define the mixture model.

First we define the mixture of the observations as well as the component BNs in the mixture model. 

The elements of `mixing rate` defines the proportion of the total number of observations that are generated from each component. The entries should sum up to 1. The number of components `n_components` in the mixture model is taken to be the number indicated by the mixing rates, but can be set to a different value if so desired.

Each Gaussain BN has `n_vars` variables and has either a linear or nonlinear mean function depending on `struct_eq_type`. The nonlinear functions are modelled using shallow NNs. The random graph prior is either set to `er` (Erdos-Renyi) or `sf` (scale-free) graphs. 

In [ ]:
# Set random seed
key = random.PRNGKey(523)

# Data settings
n_observations = 0 # total number of observations
mixing_rate = jnp.array([1.0]) # mixing rate of observations (also defines number of components)
#mixing_rate = 1/(K:=5) * jnp.ones(K) # uniform mixing rates for K components

# BN settings
n_vars = 3 # number of variables in each component BN
struct_eq_type = 'linear' # BN function class: 'linear' or 'nonlinear'
graph_type = 'sf' # Random graph structure: 'sf' (scale-free) or 'er' (Erdos-Renyi)

# Derived variables
n_components = mixing_rate.shape[0]
linear = True if struct_eq_type == 'linear' else False

### Generate ground truth BNs and synthetic data.

We generate synthetic data from ground truth BNs as specified above. `x_ind` is an observation array with {n_observations, n_vars+1}, where the last column consits of ground truth indicators for the corresponding ground truth BN. The varaibles `lik`, `component_lik`, and `graph_model` define likelihood models and the grpah prior model.

In [ ]:
from vamsl.target import make_mixture_model
from vamsl.utils import visualize_ground_truths

key, subk = random.split(key)
# Generate data (with indicators for component assignment), ground truth BNs, likelihood models, and random graph model
x_ind, ground_truth_graphs, ground_truth_thetas, lik, component_lik, graph_model = make_mixture_model(key=subk,
                                                                                                      mixing_rate=mixing_rate,
                                                                                                      n_vars=n_vars,
                                                                                                      n_observations=n_observations,
                                                                                                      graph_type=graph_type,
                                                                                                      struct_eq_type=struct_eq_type)
#visualize_ground_truths(ground_truth_graphs)
print(ground_truth_graphs)
# Remove indicator vector from data
x = x_ind[:,0:n_vars] # [n_observations, n_vars]
print('Observations shape:')
print('x: ' + str(x.shape))
print('Indicators and counts:')
print(jnp.unique(x_ind[:,n_vars], return_counts=True))

### Create VaMSL.

We create the VaMSL model and intilitialize it with `n_particles` randomly sampled particles latent $Z$ and $\Theta $ particles as well as uniform distributions over responsibilities and weights. If one wants to specify specific prior responsibilities and weights, they can be fed to `initialize_posteriors` as named variables `init_q_c` and `alphas`, of shapes `(n_observations, n_components)` and `(n_components,)` respectively. 

In [ ]:
from vamsl.inference import VaMSL

# Create VaMSL and initialize posteriors (remove indicator vecor from dataset)
vamsl = VaMSL(x=x, graph_model=graph_model, mixture_likelihood_model=lik, component_likelihood_model=component_lik)
key, subk = random.split(key)
vamsl.initialize_posteriors(key=subk, n_components=n_components, n_particles=30, linear=linear)

print('Posterior shapes:')
posts = vamsl.get_posteriors()
print('q_z:     ' + str(posts[0].shape)) # [n_components, n_particls, d, l, 2]
print('q_theta: ' + str(posts[1].shape)) if linear else print('q_theta: ' + str(len(posts[1]))) # leading dim of n_components
print('log_q_c: ' + str(posts[2].shape)) # [n_observations, n_components]
print('q_pi:    ' + str(posts[3].shape)) # [n_components,]

In [ ]:
key, subk = random.split(key)
_, false_graphs, _, _, _, _ = make_mixture_model(key=subk,mixing_rate=mixing_rate,n_vars=n_vars,n_observations=n_observations,
                                                 graph_type=graph_type,struct_eq_type=struct_eq_type)

#E = jnp.ones((n_components, n_vars, n_vars)).at[i,j,...].set(1.0) # supply individual constraints 
E = jnp.where(ground_truth_graphs, 0.99, 0.01) # supply correct soft constraints 
#E = ground_truth_graphs # supply correct hard constraints

#print(false_graphs)
vamsl.set_E(E)
vamsl.get_E()
#vamsl.elicitation_probs_to_hard_constraints(vamsl.get_E()) # check hard constraints encoding

In [ ]:
# CAVI and SVGD vars
n_cavi_updates, steps, callback_every = 0, 500, 100

# CAVI-loop
for cavi_update in range(n_cavi_updates):
    key, subk = random.split(key)
    # Optimize q(Z, \Theta)
    vamsl.update_particle_posteriors(key=subk, steps=steps, callback_every=callback_every,
                                     callback=vamsl.visualize_callback(), linear=linear)

    # Update to optimal q(c) and q(\pi)
    vamsl.update_responsibilities_and_weights()
    print(f'CAVI update number {cavi_update+1}/{n_cavi_updates}')
    
# Final CAVI update with more SVGD steps to ensure annealing unto acyclic graphs
key, subk = random.split(key)
vamsl.update_particle_posteriors(key=subk, steps=1000, callback_every=1000, callback=vamsl.visualize_callback(), linear=linear)
#vamsl.update_responsibilities_and_weights()

### Get order of components with respect to ground truths

We identify an ordering of the components with respect to the ground truth BNs by identifying which perumutation of indeces results in the highest classification accuracy.

In [ ]:
# Compute optimal ordering with respect to MAP classification accuracy
order = vamsl.identify_MAP_classification_ordering(ground_truth_indicator=x_ind[:,n_vars])
print('Optimal order:')
print(order)

### Evaluate classification performance

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print('Sums of responsibilities:')
print(jnp.sum(jnp.exp(vamsl.get_posteriors()[2]), axis=0))
print('Sum of entropy of responsibilities:')
print(jnp.sum(jnp.exp(vamsl.get_posteriors()[2])*vamsl.get_posteriors()[2]))
y_true = x_ind[:,n_vars]
y_pred = [order[k] for k in [jnp.argmax(c_i) for c_i in vamsl.get_posteriors()[2]]]
if n_components==2:
    print('Classification report:')
    print(classification_report(y_true=y_true, y_pred=y_pred, target_names=['Component 1', 'Component 2']))
confusion_matrix(y_true, y_pred)

### Evaluate structure learning performance

In [ ]:
from vamsl.metrics import expected_shd, threshold_metrics, neg_ave_log_likelihood

# Get component datasets
datas = [x_ind[x_ind[:,n_vars]==k][:,:n_vars] for k in range(n_components)]

# Loop over components and calculate metrics
for k, data, q_z_k, q_theta_k in zip(range(n_components), datas, vamsl.get_posteriors()[0], vamsl.get_posteriors()[1]):
    # Get particle distribution for component
    q_g_k = vamsl.particle_to_g_lim(q_z_k, vamsl.get_E()[order[k]])
    dist = vamsl.get_empirical(q_g_k, q_theta_k)
    
    # Calculate metrics
    eshd = expected_shd(dist=dist, g=ground_truth_graphs[order[k]])       
    auroc = threshold_metrics(dist=dist, g=ground_truth_graphs[order[k]])['roc_auc']
    #negll = neg_ave_log_likelihood(dist=dist, eltwise_log_likelihood=vamsl.eltwise_component_log_likelihood_observ, x=data.x_ho)
    negll=0
    
    print(f' Component {k+1:4d} |  E-SHD: {eshd:4.1f}    AUROC: {auroc:5.2f}    neg. LL {negll:5.2f}')

In [ ]:
vamsl.elicitation_probs_to_hard_constraints(vamsl.get_E())

### Elicitation

Incoporate elicited hard and soft edge constraints into VaMSL. Either provide an elicitation matrix, construct one by answering queries, or let a oracle simulate the process of querying an expert.

In [ ]:
from vamsl.elicitation.simulators import bernoulli_simulator
from vamsl.elicitation import edgeElicitation, graphOracle

init_queries = 0 
n_queries = 0 
random_queries=False
expert_reliability = 0

In [ ]:
E_1 = jnp.zeros((n_vars,n_vars)).at[0,1].set(1.0).at[0,2].set(1.0).at[2,1].set(1.0).at[1,0].set(-1.0).at[2,0].set(-1.0).at[1,2].set(-1.0)
E_2 = jnp.zeros((n_vars,n_vars)).at[1,0].set(1.0).at[2,0].set(1.0).at[3,0].set(1.0).at[0,1].set(-1.0).at[0,2].set(-1.0).at[0,3].set(-1.0)
visualize_ground_truth(E_1)
visualize_ground_truth(E_2)
#E_3 = jnp.zeros((n_vars,n_vars)).at[0,3].set(1.0).at[3,2].set(1.0).at[3,1].set(1.0)

E = jnp.stack([E_1, E_2], axis=0)
vamsl.set_E(E)

### Save model and data

In [ ]:
import pickle
#filename = '5_d_14_p_100_s_30_e_2000_s.p'
pickle.dump(vamsl.get_posteriors(), open(filename, 'wb'))

In [ ]:
import pickle
posts = pickle.load(open('20_d_14_p_200_s_20_e_2000_s.p', 'rb'))